'제주도 기념품샵' 카카오맵 크롤링

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
url = 'https://map.kakao.com'
driver = webdriver.Chrome()
driver.get(url)

In [3]:
# 지도 설정 이미지 클릭
img=driver.find_element(By.CLASS_NAME, 'inner_coach_layer')
img.click()

# 검색창에 '제주도 기념품' 입력
search=driver.find_element(By.ID, 'search.keyword.query')
search.send_keys("제주도 기념품")
search.send_keys(Keys.ENTER)
time.sleep(2)

# 장소 더보기 버튼 클릭
btnMore=driver.find_element(By.ID, "info.search.place.more")
btnMore.click()
time.sleep(2)

# 1페이지로 이동
driver.find_element(By.ID, "info.search.page.no1").click()

In [5]:
result  = []

# 'a()'라는 함수를 통해 새로운 창을 열고, 해당 창에서 특정 웹페이지로 이동하여 데이터 수집
def a():
    soup=bs(driver.page_source,"lxml")

    driver.execute_script('window.open("https://google.com");') # 구글 새창으로 열기
    time.sleep(2)

    driver.switch_to.window(driver.window_handles[-1]) # 새창으로 전환하기
    
    for i in range(0, 15):
        sub_url = soup.select('.contact.clickArea >.moreview')[i]['href'] # 상세보기 클릭
        driver.get(sub_url)
        time.sleep(2)

        # lxml: HTML 및 XML을 파싱하는 데 사용되는 파서 중 하나
        mini_soup=bs(driver.page_source,"lxml")

        try: 
            driver.find_element(By.CLASS_NAME, 'color_b').click()

            review_num=mini_soup.select_one('.total_evaluation > .color_b')
            review_num = int(review_num.text.strip())
            
            for j in range(int((review_num-3)/5+1)):
                more=driver.find_element(By.CLASS_NAME, "txt_more")
                more.click()
                time.sleep(2)

        except :
            review_num = 'none'

        time.sleep(2)

        mini_soup=bs(driver.page_source,"lxml")
        time.sleep(2)

        title = mini_soup.select_one(".tit_location").text        
        addr = mini_soup.select_one(".txt_address").text 
        try:
            link= mini_soup.select_one(".link_homepage").text
        except:
            link='none'
        time.sleep(2) 

        # 리뷰에 관한
        try: 
            review_list = mini_soup.select_one('.list_evaluation').select('li') 
            time.sleep(2)
            
            for review in review_list:
                try:           
                    star = review.select_one(".list_evaluation .ico_star.inner_star").attrs['style']
                    txt = review.select_one(".txt_comment > span").text                
                    result.append([title, addr, star, txt, link])
                except:
                    pass   
        except :
            star = 'none'
            txt = 'none'
            review = 'none'
            result.append([title, addr, star, txt, link])
    driver.close()
    time.sleep(2)
    driver.switch_to.window(driver.window_handles[0])

In [9]:
page = 1    # 현재 크롤링하는 페이지가 전체에서 몇 번째 페이지인지
page2 = 0   # 1 ~ 5번째 중 몇 번째인지
error_cnt = 0

while True:
    try:
        page2 += 1
        print("**", page, "**")
        
        driver.find_element(By.XPATH, f'//*[@id="info.search.page.no{page2}"]').send_keys(Keys.ENTER)
        time.sleep(2)
        a()
        shop_list = driver.find_elements(By.CSS_SELECTOR, '.placelist > .PlaceItem')
        if len(shop_list) < 15:
            break
		
        # 다음 버튼을 누를 수 없다면 마지막 페이지
        if not driver.find_element(By.XPATH, '//*[@id="info.search.page.next"]').is_enabled():
            break

        # 다섯번째 페이지까지 왔다면 다음 버튼을 누르고 page2 = 0으로 초기화
        if page2 % 5 == 0:
            driver.find_element(By.XPATH, '//*[@id="info.search.page.next"]').send_keys(Keys.ENTER)
            page2 = 0

        page += 1
    except Exception as e:
        error_cnt += 1
        print('ERROR!')
        if error_cnt > 5:
            break # 예외가 5번 이상 발생 시 무한 루프 종료 
    time.sleep(2)

driver.close()
driver.switch_to.window(driver.window_handles[0]) # 웹드라이버의 현재 창을 첫 번째 창으로 전환

** 1 **
** 2 **
** 3 **
** 4 **
** 5 **
** 6 **
** 7 **
** 8 **
** 9 **
** 10 **
** 11 **
** 12 **
** 13 **
** 14 **
** 15 **
** 16 **
** 17 **
** 18 **
** 19 **
** 20 **
** 21 **
** 22 **
** 23 **
** 24 **
** 25 **
** 26 **
** 27 **
** 28 **
ERROR!
** 28 **
ERROR!
** 28 **
ERROR!
** 28 **
ERROR!
** 28 **
ERROR!
** 28 **
ERROR!


In [16]:
result  

[['바이제주',
  '제주특별자치도 제주시\n                서해안로 626\n                 (우)63150',
  'width:100%;',
  '매장도 깨끗하고 상품도 다양하네요. 열쇠고리나 일반 책갈피도 기본 만원 이상이라서 가격적으로는 부담이 되지만 재방문 의사 있어요. 간단하게 지인이랑 커플로 몇가지 기념품 샀는데 20만원이 나오더라구요. 충동구매 조심하세요.ㅋㅋ',
  'blog.naver.com/byjeju365'],
 ['바이제주',
  '제주특별자치도 제주시\n                서해안로 626\n                 (우)63150',
  'width:20%;',
  '핸드메이드 제품은 많음근데 과자 여기서 사지말고 시장가서 사시길저도 알고싶지않았음^^',
  'blog.naver.com/byjeju365'],
 ['바이제주',
  '제주특별자치도 제주시\n                서해안로 626\n                 (우)63150',
  'width:80%;',
  '재밌고 흥미로운 아이템  많던곳',
  'blog.naver.com/byjeju365'],
 ['바이제주',
  '제주특별자치도 제주시\n                서해안로 626\n                 (우)63150',
  'width:100%;',
  '',
  'blog.naver.com/byjeju365'],
 ['바이제주',
  '제주특별자치도 제주시\n                서해안로 626\n                 (우)63150',
  'width:80%;',
  '종류가 많음.',
  'blog.naver.com/byjeju365'],
 ['바이제주',
  '제주특별자치도 제주시\n                서해안로 626\n                 (우)63150',
  'width:100%;',
  '',
  'blog.naver.com/byjeju365'],
 [

In [17]:
souvenir = pd.DataFrame(result)

In [18]:
souvenir.head(30)

,0,1,2,3,4
0,바이제주,제주특별자치도 제주시\n 서해안로 626\n ...,width:100%;,매장도 깨끗하고 상품도 다양하네요. 열쇠고리나 일반 책갈피도 기본 만원 이상이라서 ...,blog.naver.com/byjeju365
1,바이제주,제주특별자치도 제주시\n 서해안로 626\n ...,width:20%;,핸드메이드 제품은 많음근데 과자 여기서 사지말고 시장가서 사시길저도 알고싶지않았음^^,blog.naver.com/byjeju365
2,바이제주,제주특별자치도 제주시\n 서해안로 626\n ...,width:80%;,재밌고 흥미로운 아이템 많던곳,blog.naver.com/byjeju365
3,바이제주,제주특별자치도 제주시\n 서해안로 626\n ...,width:100%;,,blog.naver.com/byjeju365
4,바이제주,제주특별자치도 제주시\n 서해안로 626\n ...,width:80%;,종류가 많음.,blog.naver.com/byjeju365
5,바이제주,제주특별자치도 제주시\n 서해안로 626\n ...,width:100%;,,blog.naver.com/byjeju365
6,바이제주,제주특별자치도 제주시\n 서해안로 626\n ...,width:80%;,해안가도로를 따라 용두암으로 향하던 도중에 접했는데...아기자기한 소품들과 더불어서...,blog.naver.com/byjeju365
7,바이제주,제주특별자치도 제주시\n 서해안로 626\n ...,width:80%;,제주소품 선물가게로는 규모가 상당히 큰편입니다.이것저것 없는게 없는..2층에는 바다...,blog.naver.com/byjeju365
8,바이제주,제주특별자치도 제주시\n 서해안로 626\n ...,width:100%;,야간에 방문했는데 정말 소품 볼 것 많음. 아기자기하고 귀여운 것들이 많음. 자그마...,blog.naver.com/byjeju365
9,바이제주,제주특별자치도 제주시\n 서해안로 626\n ...,width:100%;,"다양한 작가들의 여행기념품들이 모여있어서 좋았고,제주도만의 먹거리 기념품들도 종류별...",blog.naver.com/byjeju365


In [19]:
souvenir.tail(30)

,0,1,2,3,4
2217,제주한잔 한화리조트 제주점,제주특별자치도 제주시\n 명림로 575-107\n ...,width:80%;,,www.instagram.com/jeju.hanjan
2218,제주한잔 한화리조트 제주점,제주특별자치도 제주시\n 명림로 575-107\n ...,width:100%;,제주술을 전문적으로 파는 곳입니다. 양조장과 술에 대한 정보를 꼼꼼하게 설명해주고 ...,www.instagram.com/jeju.hanjan
2219,제주한잔 한화리조트 제주점,제주특별자치도 제주시\n 명림로 575-107\n ...,width:100%;,감사하는 마음으로 리뷰를 쓰게되는 곳,www.instagram.com/jeju.hanjan
2220,제주한잔 한화리조트 제주점,제주특별자치도 제주시\n 명림로 575-107\n ...,width:100%;,제주에서 갔던 곳 중 가장 좋았어요 맑은바당 추천 합니다,www.instagram.com/jeju.hanjan
2221,제주한잔 한화리조트 제주점,제주특별자치도 제주시\n 명림로 575-107\n ...,width:100%;,제주에서 만들어진 다양한 주류들을 만날 수 있습니다. 내부인테리어도 깔끔하고 직원분...,www.instagram.com/jeju.hanjan
2222,제주한잔 한화리조트 제주점,제주특별자치도 제주시\n 명림로 575-107\n ...,width:100%;,체험해보기 어려운 전통주들을 샘플러로 먹을 수 있어서 너무 좋았고 맛도 좋아서 구매...,www.instagram.com/jeju.hanjan
2223,제주한잔 한화리조트 제주점,제주특별자치도 제주시\n 명림로 575-107\n ...,width:100%;,제주전통주 맛보기 좋은 곳.,www.instagram.com/jeju.hanjan
2224,제주한잔 한화리조트 제주점,제주특별자치도 제주시\n 명림로 575-107\n ...,width:100%;,제주의 다양한 전통주를 사람들에게 사다주려고 했는데 막상 제가 맛보지 않으니 고르기...,www.instagram.com/jeju.hanjan
2225,제주한잔 한화리조트 제주점,제주특별자치도 제주시\n 명림로 575-107\n ...,width:100%;,제주 술은 다 여기있는듯~제주 술 박물관도 갔다왔지만 여기면 될거같네요~,www.instagram.com/jeju.hanjan
2226,제주한잔 한화리조트 제주점,제주특별자치도 제주시\n 명림로 575-107\n ...,width:60%;,은근 내부 넓고 메뉴 많음동네술집 좋아요,www.instagram.com/jeju.hanjan


In [22]:
# souvenir 문자열의 문자 수
len(souvenir)

2247

In [21]:
df=pd.DataFrame(souvenir) # 데이터프레임으로 만들기
df.to_csv("제주도 기념품샵_크롤링 결과.csv")

전처리

In [ ]:
# 데이터 불러오기
df=pd.read_csv("./제주도 기념품샵_크롤링 결과.csv")
df.head()

In [ ]:
# 불필요한 컬럼 삭제
df.drop(columns=['Column1'], inplace=True) #원본에 그대로 적용

# 컬럼명 변경
df.rename(columns={'Column2':'이름'}, inplace=True)
df.rename(columns={'Column3':'주소'}, inplace=True)
df.rename(columns={'Column4':'별점'}, inplace=True)
df.rename(columns={'Column5':'리뷰'}, inplace=True)
df.rename(columns={'Column6':'홈페이지'}, inplace=True)

df.head(10)

In [ ]:
df['주소'] = df['주소'].str.replace('\n', ' ')

df['주소']=df['주소'].str.replace('                 ', ' ')

In [ ]:
split_addr=df['주소'].str.split("(", expand=True) # 분할된 결과를 여러 열로 확장하여 반환
split_addr

In [ ]:
# 기존 df에 새로운 열로 추가
df['주소3'] = split_addr[1]
df['주소'] = split_addr[0]

# 불필요한 컬럼 삭제
df.drop(columns=['주소3'], inplace=True) # 원본에 그대로 적용

# index를 포함하지 않고 데이터 저장
df.to_csv("제주도 기념품샵 목록.csv", index=False)

# 데이터 불러오기
df=pd.read_csv("./제주도 기념품샵 목록.csv")
df.head()

In [ ]:
df['별점'].replace(['width:100%;', 'width:80%;', 'width:60%;', 'width:40%;', 'width:20%;', 'none'],
              ['5', '4', '3', '2', '1', ''], inplace=True)

df['리뷰'].replace(['none'],[''], inplace=True) # none -> 공백
df['리뷰'] = df['리뷰'].replace('none', '')

df['홈페이지'].replace(['none'],[''], inplace=True)

# 별점, 리뷰, 홈페이지 컬럼의 빈 부분을 공백으로 채우기
df['별점']=df['별점'].fillna('')
df['리뷰']=df['리뷰'].fillna('')
df['홈페이지']=df['홈페이지'].fillna('')

In [ ]:
# index를 포함하지 않고 데이터 저장
df.to_csv("제주도 기념품샵 목록.csv", index=False)

카카오 API 위도경도

In [ ]:
# API 키 가져오기
with open('C:/workspace/kakao.txt') as file:
    road_key = file.read()

# URL 만들기
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '제주특별자치도 제주시 서해안로 626'
url = f'{base_url}?query={quote(addr)}'

# Header : Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {road_key}'}

# Kakao Local API 호출하여 결과 가져오기
result = requests.get(url, headers=header).json()
result

In [ ]:
lat = result['documents'][0]['y'] # 위도(latitude)
lng = result['documents'][0]['x'] # 경도(latitude)
dong=result['documents'][0]['address']['region_3depth_name']

In [ ]:
# 제주도 기념품샵 정보 완성하기
df = pd.read_csv("./제주도 기념품샵 목록.csv")
df

In [ ]:
lat_list, lng_list, dong_list = [], [], [] # 빈 리스트 생성. 위도와 경도 값 저장
for i in df.index: # df의 각 행에 대해 반복
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json() # API에 요청을 보내고 응답을 JSON 형식으로 받음
    try:
        lat_list.append(result['documents'][0]['y']) # 위도(y) 추출하여 lat_list에 추가
        lng_list.append(result['documents'][0]['x']) # 경도(x) 추출하여 lng_list에 추가
        dong_list.append(result['documents'][0]['address']['region_3depth_name'])
    except:
        print(df.이름[i])  
# try-except 구문: try블록 실행 중 예외 발생 시 except블록 실행

In [ ]:
df[df['이름']=='선물고팡 제주공항점']

# map함수를 활용해 선물고팡 제주공항점 주소 변경
df.loc[203:321, '주소'] = df.loc[203:321]['주소'].map({'제주특별자치도 제주시 월성로 15 1,2층 ':'제주특별자치도 제주시 월성로 15 1'})

In [ ]:
df[df['이름']=='더아일랜더']

# 더아일랜더 주소 변경
df.loc[1172:1220, '주소'] = df.loc[1172:1220]['주소'].map({'제주특별자치도 제주시 관덕로4길 7 1,2층 ':'제주특별자치도 제주시 관덕로4길 7 1'})

In [ ]:
df[df['이름']=='제주고래감성소품']

# 제주고래감성소품 주소 변경
df.loc[1655:1656, '주소'] = df.loc[1655:1656]['주소'].map({'제주특별자치도 제주시 애월읍 애월해안로 390 1,2층 ':'제주특별자치도 제주시 애월읍 애월해안로 390 1'})

In [ ]:
# 수정된 데이터로 다시 위도경도 받아오기
lat_list, lng_list, dong_list = [], [], [] # 빈 리스트 생성. 위도와 경도 값 저장
for i in df.index: # df의 각 행에 대해 반복
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json() # API에 요청을 보내고 응답을 JSON 형식으로 받음
    try:
        lat_list.append(result['documents'][0]['y']) # 위도(y) 추출하여 lat_list에 추가
        lng_list.append(result['documents'][0]['x']) # 경도(x) 추출하여 lng_list에 추가
        dong_list.append(result['documents'][0]['address']['region_3depth_name'])
    except:
        print(df.이름[i])
# try-except 구문: try블록 실행 중 예외 발생 시 except블록 실행

In [ ]:
# '선물고팡 제주공항점', '두몰', '더아일랜더', '제주고래감성소품', '제라몽' 행 제거
df=df[df.이름 != '선물고팡 제주공항점']
df=df[df.이름 != '도두몰']
df=df[df.이름 != '더아일랜더']
df=df[df.이름 != '제주고래감성소품']
df=df[df.이름 != '제라몽']

In [ ]:
df['위도']=lat_list
df['경도']=lng_list
df['주소2']=dong_list

# 제주도 기념품샵 위도 경도 동 리스트
lat_list
lng_list
dong_list

In [ ]:
df.to_csv("제주도 기념품샵 목록.csv", index=False)

지도 시각화

In [ ]:
df = pd.read_csv('./제주도 기념품샵 목록.csv')      
df

In [ ]:
# folium.Map() 함수 사용하여 지도 객체 생성하기
# df의 위도와 경도 열의 평균값인 위치에 해당하는 지도를 생성하고 확대 수준을 10으로 설정
map=folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=10)
map

In [ ]:
# 제주시(blue)-서귀포시(green) 구분해서 folium.Marker
tiles = "http://mt0.google.com/vt/lyrs=m&hl=ko&x={x}&y={y}&z={z}" #구글지도 타일 설정
attr= "Google" #속성 설정
map = folium.Map(location=[df['위도'].mean(), df['경도'].mean()], #folium을 사용하여 지도 초기화
                 zoom_start=10, tiles=tiles, attr=attr)
for i in df.index: #df의 각 행을 반복
    if '제주시' in df.loc[i, '주소']:
        folium.Marker(
            location=[df.loc[i, '위도'], df.loc[i, '경도']],
            tooltip=df.loc[i, '이름'], #마커에 마우스를 올렸을 때 표시되는 툴팁으로 이름 표시
            popup=folium.Popup(df.loc[i, '주소'], max_width=200),
            icon=folium.Icon(color='green', icon='glyphicon-gift')
        ).add_to(map)
    else:
        folium.Marker(
            location=[df.loc[i, '위도'], df.loc[i, '경도']],
            tooltip=df.loc[i, '이름'], #마커에 마우스를 올렸을 때 표시되는 툴팁으로 이름 표시
            popup=folium.Popup(df.loc[i, '주소'], max_width=200),
            icon=folium.Icon(color='blue', icon='glyphicon-gift')
        ).add_to(map)

map        